In [41]:
from pptx import Presentation
import json
import yaml

prs = Presentation('../Oracle_PPT-template_FY26_blank.pptx')


In [31]:
layouts_list = []

for i, layout in enumerate(prs.slide_layouts):    
    # if layout.name not in ["Light - Title_Pillar","Light - Divider","Light - Single metric","Light - Thank You","Light - Agenda Table","Light - Title 1 Column","Light - Title/Subtitle 1 Column"]:
    #     continue
    print(layout.name)
    shortname = layout.name.split("-")[-1].strip()
    mode = "dark" if "Dark" in layout.name else "light"
    layout_dict = {
            "index": i,
            "layout_name": layout.name,
            "mode": mode,
            "shortname": shortname,
            "description": "",
            "placeholders": []
    }
    
    for j,placeholder in enumerate(layout.placeholders):
        # if placeholder.name.startswith("Slide Number") or placeholder.name.startswith("Footer") or placeholder.name.startswith("Date"):
        #     continue
        layout_dict["placeholders"].append(
            {
            "index":j,
            "place_holder_index":placeholder.placeholder_format.idx,
            "placeholder_name": placeholder.name,
            "placeholder_id": placeholder.shape_id,
            "alias": placeholder.name,
            "description": ""
            }
        )
    
    layouts_list.append(layout_dict)

yaml_data = yaml.dump({"layouts": layouts_list}, sort_keys=False)
with open("assets/layouts_all.yaml", "w") as f:
    f.write(yaml_data)  


Dark - Title_Pillar
Light - Title_Pillar
Dark - Title_2
Light - Title_2
Title-full-image
Title/Cover_3
Title/Cover_4
Title/Cover_5
Cover/Title_6
Title/Cover_7
Title/Cover_8
Title/Cover_9
Title/Cover_10
Section-full-image
Light - Divider_Image
Dark- Divider_Image
Light - Divider
Dark - Divider
Divider_3
Divider_4
Divider_5
Divider_6
Divider_7
Divider_8
Divider_9
Divider_10
Dark - Impact Statement
Light - Impact Statement
Dark - Quote with Data Texture Background
Light - Quote with Data Texture Background
Dark - Quote, Picture, and logo
Light - Quote, Picture, and logo
Dark - Single metric
Light - Single metric
Dark - Thank You
Light - Thank You
Dark - Safe harbor_Pillar
Light - Safe harbor_Pillar
Note on safe harbor/forward looking
Speaker_1_Name
Speaker_2_Name
Speaker_3_Name
Speaker_4_Name
Dark - Agenda
Light - Agenda
Dark - Agenda Table
Light - Agenda Table
Multi Statement – Dark
Multi Statement – Light
Speaker_1
Speaker_1-Profile
Speaker_2
Speaker_3
Speaker_4
Speaker_6
Speaker_10
Tit

In [39]:
with open("assets/layouts_all.yaml", "r") as f:
    yaml_data = f.read()
layouts = yaml.safe_load(yaml_data)["layouts"]

layout_description = {}

for layout in layouts:
    shortname = layout["shortname"]
    mode = layout["mode"]
    if shortname not in layout_description.keys():
        layout_description[shortname] = {
            "layout_name": {mode:layout["layout_name"]},
            "index": {mode: layout["index"]},
            "alias": shortname,
            "description": layout["description"]
        }

        placeholders = {}
        for placeholder in layout["placeholders"]:            
            placeholders[placeholder["index"]] = {
                "name":placeholder["placeholder_name"],
                "alias":placeholder["alias"],
                "description":placeholder["description"],
                "place_holder_index":{mode:placeholder["place_holder_index"]}
                }
        layout_description[shortname]["placeholders"] = placeholders
    else:   
        layout_description[shortname]["index"][mode] = layout["index"]
        for placeholder in layout["placeholders"]:            
            try:
                layout_description[shortname]["layout_name"][mode] = layout["layout_name"]
                layout_description[shortname]["placeholders"][placeholder["index"]]["place_holder_index"][mode] = placeholder["place_holder_index"]
            except Exception as e:
                print(e)
                print(layout_description[shortname])
                print(layout)
                break

with open("assets/layouts_description.yaml", "w") as f:
    f.write(yaml.dump(layout_description, sort_keys=False))

In [42]:
# update_layout_description
with open("assets/layouts_description_edit.yaml", "r") as f:
    yaml_data = f.read()
template = yaml.safe_load(yaml_data)

with open("assets/layouts_description.yaml", "r") as f:
    yaml_data = f.read()
layouts = yaml.safe_load(yaml_data)

for k,v in layouts.items():
    if k in template.keys():
        layouts[k]["alias"] = template[k]["alias"]
        layouts[k]["description"] = template[k]["description"]
        layouts[k]["placeholders"] = template[k]["placeholders"]
    else:
        layouts[k]["disabled"] = True

with open("assets/layouts_description.yaml", "w") as f:
    f.write(yaml.dump(layouts, sort_keys=False))





In [22]:
# 生成提示词

with open("assets/layouts_description_template.yaml", "r") as f:
    yaml_data = f.read()
layouts_template = yaml.safe_load(yaml_data)

layout_dict = {}
for k,v in layouts_template.items():
    placeholder_dict = {}        
    if v.get("disabled", False):
        continue
    layout_alias = v["alias"]
    if v.get("placeholders", []):
        for pk,pv in v["placeholders"].items():
            if pv.get("disabled", False):
                continue
            alias = pv["alias"]
            placeholder_dict[alias] = pv["description"]
        layout_dict[layout_alias] = {
                "description":v["description"],
                "placeholders": placeholder_dict
            }
print(json.dumps(layout_dict, indent=4))

{
    "Title_Pillar": {
        "description": "The main title page",
        "placeholders": {
            "Title": "title for the pressentation, one line, max 50 characters",
            "Subhead": "subtitle for the pressentation, one line, max 40 characters"
        }
    },
    "Divider_Image": {
        "description": "narrow divider page for different sections, with main title and subtitle",
        "placeholders": {
            "Title": "main title for the section, very short one line, max 30 characters",
            "Subhead": "subtitle for the section, very short one line, max 50 characters"
        }
    },
    "Divider": {
        "description": "wide divider page for different sections, with main title and subtitle",
        "placeholders": {
            "Title": "main title for the section, one line, max 50 characters",
            "Subhead": "subtitle for the section, one line, max 100 characters"
        }
    },
    "Impact Statement": {
        "description": "impact s

In [29]:
# 生成索引
with open("assets/layouts_description_template.yaml", "r") as f:
    yaml_data = f.read()
layouts_template = yaml.safe_load(yaml_data)

reverse_index = {}
for k,v in layouts_template.items():
    alias = v["alias"]
    index = v["index"]
    reverse_p_index = {}
    if "placeholders" in v:
        for pk,pv in v["placeholders"].items():
            for ik,iy in pv["place_holder_index"].items():
                p_alias = pv["alias"]
                p_index = pv["place_holder_index"]
                reverse_p_index[p_alias] = p_index
    reverse_index[alias] = {"layout_index":index, "placeholders": reverse_p_index}

reverse_index

{'Title_Pillar': {'layout_index': {'dark': 0, 'light': 1},
  'placeholders': {'Title': {'dark': 0, 'light': 0},
   'Subhead': {'dark': 33, 'light': 33},
   'Text Field': {'dark': 35, 'light': 35},
   'Text Field 2': {'dark': 34, 'light': 34},
   'Slide Number Placeholder 4': {'dark': 12, 'light': 12},
   'Footer Placeholder 3': {'dark': 36, 'light': 10},
   'Date Placeholder 2': {'dark': 10, 'light': 36}}},
 'Divider_Image': {'layout_index': {'light': 2, 'dark': 3},
  'placeholders': {'Title': {'light': 0, 'dark': 34},
   'Picture Placeholder 15': {'light': 34, 'dark': 0},
   'Slide Number Placeholder 7': {'light': 12, 'dark': 12},
   'Footer Placeholder 4': {'light': 11, 'dark': 11},
   'Date Placeholder 2': {'light': 10, 'dark': 10},
   'Subhead': {'light': 33, 'dark': 33}}},
 'Divider': {'layout_index': {'light': 4, 'dark': 5},
  'placeholders': {'Title': {'light': 0, 'dark': 0},
   'Subhead': {'light': 33, 'dark': 33},
   'Slide Number Placeholder 7': {'light': 12, 'dark': 12},
   